# Step (n-2): Generate Feature Vector

In [1]:
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from peoplefinder import PeopleFinder
from operator import attrgetter
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from tqdm import tqdm

def subject_match(engine,A):
    query = ("SELECT bill_type,bill_number,status,result FROM allbills4 WHERE (subjects LIKE '%%{" + A
                     + ",%%' OR subjects LIKE '%%,"+A+",%%' OR subjects LIKE '%%,"+A+"}%%');")
    #use query and put in DataFrame
    foo = pd.read_sql_query(query,engine)
    return foo

def get_subject_features(engine):
    query = "SELECT * FROM subs_morebills_hair"
    foo = pd.read_sql_query(query, engine)
    return foo

def clean_subject(a):
    try:
        quoteloc = a.index("'") #may get issues with multiple single quotes in one topic.
        a = a[:quoteloc]+"'"+a[quoteloc:]
    except:
        pass #no quotes found
    #need to add " if there are any spaces in the subject
    try:
        quoteloc = a.index(" ") #will go to except if there are no spaces in the subject
        a = '\"'+a+'\"'
    except:
        pass #no spaces found
    return a

def remove_paren(a):
    try:
        parenloc = a.index("(")
        a = a[:parenloc]+a[parenloc+1:]
    except:
        pass
    try:
        parenloc = a.index(")")
        a = a[:parenloc]+a[parenloc+1:]
    except:
        pass
    return a

Populating the interactive namespace from numpy and matplotlib


In [2]:
#set up postgresql engine
dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)
conn = psycopg2.connect("dbname='legislatr' user='lordluen' host='localhost'")
cur = conn.cursor()

In [3]:
#load in subject features
sub_feat_temp = get_subject_features(engine)
sub_feat = pd.DataFrame()
sub_feat['subjects'] = sub_feat_temp.subjects
sub_feat['q_subjects'] = sub_feat['subjects'].map( lambda x: clean_subject(x)) #subjects for querying.
sub_feat['df_subjects'] = sub_feat['subjects'].map( lambda x:remove_paren(x))

In [4]:
print(sub_feat.subjects.size)
sub_feat.head(3)

406


,subjects,q_subjects,df_subjects
0,Access Board,"""Access Board""",Access Board
1,Administrative Conference of the U.S.,"""Administrative Conference of the U.S.""",Administrative Conference of the U.S.
2,Administrative law and regulatory procedures,"""Administrative law and regulatory procedures""",Administrative law and regulatory procedures


In [5]:
#create a dataframe will bill in first column, pass or fail in second column (will need to clean), and features in reamaining columns
#first get a list of bills.
#bills_subjects = pd.read_sql("SELECT subjects FROM allbills;",engine)
bills_subjects = pd.read_sql("SELECT subjects FROM allbills4;",engine)
#bills_features = pd.read_sql("SELECT bill_type,bill_number,status,result FROM allbills;",engine)
bills_features_subs = pd.read_sql("SELECT bill_type,num_amends,congress,bill_number,status,result,final_result FROM allbills4;",engine)
print(bills_features_subs['bill_type'].size)
bills_features_subs.head(3)
#bills_subjects.head(3)

38340


,bill_type,num_amends,congress,bill_number,status,result,final_result
0,s,0,113,1870,REPORTED,2,0
1,s,0,113,125,REFERRED,2,0
2,s,0,113,1027,REFERRED,2,0


In [6]:
bills_subjects.head(3)

,subjects
0,"{""Administrative law and regulatory procedures..."
1,"{""Aquatic ecology"",""Congressional oversight"",F..."
2,"{""Congressional oversight"",""Department of Heal..."


In [7]:
#CELL NO LONGER IN USE
#second make new dataframe with just type, number, and result.
#bills_features = pd.DataFrame()
#bills_features[['type','number','result']] = bills[['type','number','result']]

In [8]:
#bills_features.head(3)

In [9]:
#query type and number for bills that match each subject, map into df.
#loop over subjects.
for i in tqdm(range(0,sub_feat['subjects'].size)):
#get subject.
#qsub = sub_feat['q_subjects'].iloc[i] #pull out subject to search for
#currently using a postgresql query to get bill info, might be faster to search the dataframe I already have loaded.
#matches = subject_matches(engine,qsub) #get bill identifiers that have those subjects
    qsub = sub_feat['subjects'].iloc[i] #pull out subject to search for
    csub = sub_feat['df_subjects'].iloc[i] #subject to be column name
    boolcount = bills_subjects.subjects.str.contains(qsub, na=False)
    bills_features_subs[csub] = boolcount.map({False:0,True:1}).astype(int) #subjects for querying.

100%|██████████| 406/406 [00:11<00:00, 34.25it/s]


In [10]:
#check that it worked.
#print(bills_features_subs.loc[0][bills_features_subs.loc[0] == 1])
#print(bills_subjects['subjects'].iloc[0])

In [11]:
#print(np.max(bills_features_subs['num_amends']))
#bills_features.head(3)

In [12]:
#check number of success vs fail
#print(bills_features[bills_features['final_result'] == 0]['bill_number'].size)
#print(bills_features[bills_features['final_result'] == 1]['bill_number'].size)
#print(bills_features[bills_features['final_result'] == 2]['bill_number'].size)
#print(np.unique(bills_features[bills_features['final_result'] == 2]['congress']))

In [13]:
#generate senator feature matrix.

In [14]:
#%pylab inline
#import pandas as pd
#import contributions
#import votes
#import bills
#import sqlCommands
#from peoplefinder import PeopleFinder
#from operator import attrgetter
#from sqlalchemy import create_engine
#from sqlalchemy_utils import database_exists, create_database
#import psycopg2
#from tqdm import tqdm

##set up postgresql engine
#dbname = 'legislatr'
#engine = sqlCommands.get_engine(dbname)
#conn = psycopg2.connect("dbname='legislatr' user='lordluen' host='localhost'")
#cur = conn.cursor()

In [15]:
#read in sponsors
#load in subject features
#sub_feat_temp = get_sponsor_features(engine)
spon_feat = pd.read_sql_table('sponsor_list',engine)
#sub_feat['subjects'] = sub_feat_temp.subjects
#sub_feat['q_subjects'] = sub_feat['subjects'].map( lambda x: clean_subject(x)) #subjects for querying.
#sub_feat['df_subjects'] = sub_feat['subjects'].map( lambda x:remove_paren(x))
spon_feat

,index,sponsor
0,0,"Rep,""Abercrombie, Neil"",HI"
1,1,"Rep,""Abraham, Ralph Lee"",LA"
2,2,"Rep,""Ackerman, Gary L."",NY"
3,3,"Rep,""Adams, Alma S."",NC"
4,4,"Rep,""Adams, Sandy"",FL"
5,5,"Rep,""Aderholt, Robert B."",AL"
6,6,"Rep,""Adler, John H."",NJ"
7,7,"Rep,""Aguilar, Pete"",CA"
8,8,"Rep,""Akin, W. Todd"",MO"
9,9,"Rep,""Alexander, Rodney"",LA"


In [16]:
bills_sponsors = pd.read_sql("SELECT sponsor,cosponsors FROM allbills4;",engine)
bills_sponsors['allspons'] = np.asarray(bills_sponsors['sponsor'])+np.asarray(bills_sponsors['cosponsors'])
#bills_features = pd.read_sql("SELECT bill_type,bill_number,status,result FROM allbills;",engine)
bills_features_legis = pd.read_sql("SELECT bill_type,num_amends,congress,bill_number,status,result,final_result FROM allbills4;",engine)
#print(bills_features['bill_type'].size)
#bills_features.head(3)
bills_sponsors.head(3)

,sponsor,cosponsors,allspons
0,"{Sen,""Baucus, Max"",MT}",{},"{Sen,""Baucus, Max"",MT}{}"
1,"{Sen,""Brown, Sherrod"",OH}","{{Sen,""Casey, Robert P., Jr."",PA},{Sen,""Franke...","{Sen,""Brown, Sherrod"",OH}{{Sen,""Casey, Robert ..."
2,"{Sen,""Kirk, Mark Steven"",IL}","{{Sen,""Hatch, Orrin G."",UT},{Sen,""Johnson, Tim...","{Sen,""Kirk, Mark Steven"",IL}{{Sen,""Hatch, Orri..."


In [17]:
#i = 0
#qsub = spon_feat['sponsor'].iloc[i]
#boolcount = bills_sponsors.allspons.str.contains(qsub, na=False)
#print(boolcount[boolcount == True][0:10])
#print(qsub)
#print(bills_sponsors['allspons'].iloc[13146])

In [18]:
#query type and number for bills that match each subject, map into df.
#loop over subjects.
for i in tqdm(range(0,spon_feat['sponsor'].size)):
#get subject.
#qsub = sub_feat['q_subjects'].iloc[i] #pull out subject to search for
#currently using a postgresql query to get bill info, might be faster to search the dataframe I already have loaded.
#matches = subject_matches(engine,qsub) #get bill identifiers that have those subjects
    qsub = spon_feat['sponsor'].iloc[i] #pull out subject to search for
    csub = qsub #subject to be column name
    boolcount = bills_sponsors.allspons.str.contains(qsub, na=False)
    bills_features_legis[csub] = boolcount.map({False:0,True:1}).astype(int) #subjects for querying.

100%|██████████| 869/869 [00:31<00:00, 27.30it/s]


In [19]:
#repeat for committees
#%pylab inline
#import pandas as pd
#import contributions
#import votes
#import bills
#import sqlCommands
#from peoplefinder import PeopleFinder
#from operator import attrgetter
#from sqlalchemy import create_engine
#from sqlalchemy_utils import database_exists, create_database
#import psycopg2
#from tqdm import tqdm

##set up postgresql engine
#dbname = 'legislatr'
#engine = sqlCommands.get_engine(dbname)
#conn = psycopg2.connect("dbname='legislatr' user='lordluen' host='localhost'")
#cur = conn.cursor()

In [20]:
#read in sponsors
#load in subject features
#sub_feat_temp = get_sponsor_features(engine)
comm_feat = pd.read_sql_table('committee_list',engine)
#sub_feat['subjects'] = sub_feat_temp.subjects
#sub_feat['q_subjects'] = sub_feat['subjects'].map( lambda x: clean_subject(x)) #subjects for querying.
#sub_feat['df_subjects'] = sub_feat['subjects'].map( lambda x:remove_paren(x))
comm_feat

,index,committee
0,0,HLIG
1,1,HSAG
2,2,HSAP
3,3,HSAS
4,4,HSBA
5,5,HSBU
6,6,HSED
7,7,HSFA
8,8,HSGO
9,9,HSHA


In [21]:
bills_comms = pd.read_sql("SELECT committees FROM allbills4;",engine)
#bills_sponsors['allspons'] = np.asarray(bills_sponsors['sponsor'])+np.asarray(bills_sponsors['cosponsors'])
#bills_features = pd.read_sql("SELECT bill_type,bill_number,status,result FROM allbills;",engine)
bills_features_comms = pd.read_sql("SELECT bill_type,num_amends,congress,bill_number,status,result,final_result FROM allbills4;",engine)
#print(bills_features['bill_type'].size)
#bills_features.head(3)
bills_comms.head(3)

,committees
0,{SSFI}
1,{SSEV}
2,{SSHR}


In [22]:
#query type and number for bills that match each subject, map into df.
#loop over subjects.
for i in tqdm(range(0,comm_feat['committee'].size)):
#get subject.
#qsub = sub_feat['q_subjects'].iloc[i] #pull out subject to search for
#currently using a postgresql query to get bill info, might be faster to search the dataframe I already have loaded.
#matches = subject_matches(engine,qsub) #get bill identifiers that have those subjects
    qsub = comm_feat['committee'].iloc[i] #pull out subject to search for
    csub = qsub #subject to be column name
    boolcount = bills_comms.committees.str.contains(qsub, na=False)
    bills_features_comms[csub] = boolcount.map({False:0,True:1}).astype(int) #subjects for querying.

100%|██████████| 41/41 [00:00<00:00, 49.29it/s]


In [23]:
bills_features_subs.describe()

,num_amends,congress,result,final_result,Access Board,Administrative Conference of the U.S.,Administrative law and regulatory procedures,Afghanistan,Africa,Agricultural conservation and pollution,...,Water use and supply,Watersheds,West Bank,Wetlands,Women's health,Women's rights,World health,Wyoming,Yemen,Zimbabwe
count,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,...,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000
mean,0.505556,112.392123,1.966927,0.455608,0.000756,0.000600,0.129291,0.006364,0.006442,0.005738,...,0.022144,0.008972,0.001095,0.005216,0.014606,0.003678,0.004799,0.002556,0.001252,0.000626
std,10.512045,1.108746,0.190413,0.821498,0.027493,0.024486,0.335526,0.079522,0.080006,0.075534,...,0.147154,0.094298,0.033080,0.072038,0.119972,0.060533,0.069110,0.050494,0.035361,0.025012
min,0.000000,111.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,111.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,112.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,113.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,621.000000,114.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
bills_features_legis.describe()

,num_amends,congress,result,final_result,"Rep,""Abercrombie, Neil"",HI","Rep,""Abraham, Ralph Lee"",LA","Rep,""Ackerman, Gary L."",NY","Rep,""Adams, Alma S."",NC","Rep,""Adams, Sandy"",FL","Rep,""Aderholt, Robert B."",AL",...,"Sen,""Udall, Tom"",NM","Sen,""Vitter, David"",LA","Sen,""Voinovich, George V."",OH","Sen,""Walsh, John E."",MT","Sen,""Warner, Mark R."",VA","Sen,""Warren, Elizabeth"",MA","Sen,""Webb, Jim"",VA","Sen,""Whitehouse, Sheldon"",RI","Sen,""Wicker, Roger F."",MS","Sen,""Wyden, Ron"",OR"
count,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.00000,38340.000000,38340.000000,38340.000000,...,38340.00000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000
mean,0.505556,112.392123,1.966927,0.455608,0.006260,0.005556,0.01252,0.004512,0.003391,0.010850,...,0.01951,0.027856,0.005138,0.003599,0.013276,0.012598,0.004617,0.031612,0.019588,0.026395
std,10.512045,1.108746,0.190413,0.821498,0.078872,0.074329,0.11119,0.067022,0.058132,0.103599,...,0.13831,0.164562,0.071498,0.059887,0.114455,0.111532,0.067789,0.174967,0.138581,0.160310
min,0.000000,111.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,111.000000,2.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,112.000000,2.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,113.000000,2.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,621.000000,114.000000,2.000000,2.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
bills_features_comms.describe()

,num_amends,congress,result,final_result,HLIG,HSAG,HSAP,HSAS,HSBA,HSBU,...,SSEG,SSEV,SSFI,SSFR,SSGA,SSHR,SSJU,SSRA,SSSB,SSVA
count,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,...,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000,38340.000000
mean,0.505556,112.392123,1.966927,0.455608,0.004434,0.022300,0.010850,0.033516,0.053182,0.014345,...,0.039411,0.021648,0.106625,0.010694,0.028978,0.046427,0.037924,0.003730,0.005164,0.016458
std,10.512045,1.108746,0.190413,0.821498,0.066441,0.147661,0.103599,0.179982,0.224399,0.118911,...,0.194572,0.145535,0.308640,0.102858,0.167746,0.210410,0.191015,0.060959,0.071678,0.127230
min,0.000000,111.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,111.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,112.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,113.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,621.000000,114.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
sqlCommands.write_to_database(dbname,'features_subs',bills_features_subs,engine)

In [29]:
sqlCommands.write_to_database(dbname,'features_legis',bills_features_legis,engine)

In [30]:
sqlCommands.write_to_database(dbname,'features_comms',bills_features_comms,engine)

In [27]:
#save results to database.
#sqlCommands.write_to_database(dbname,'features',bills_features,engine) #this is large, give it a few minutes
#too many features, need to split into three feature matrices
#sqlCommands.write_to_database(dbname,'features_subs',bills_features_subs,engine)
#sqlCommands.write_to_database(dbname,'features_legis',bills_features_legis,engine)
#sqlCommands.write_to_database(dbname,'features_comms',bills_features_comms,engine)

KeyboardInterrupt: 

# STOP HERE!

In [46]:
print(np.sum(bills_features.outcome))
print(bills_features.outcome.size)

1054
1080


It appears the category passage is biased towards passing bills......
May need to redo everything using different categories.

In [47]:
#playing with other categories ['amendment' 'recommit' 'passage-suspension' 'procedural' 'passage' 'cloture']
bills = pd.read_sql("SELECT * FROM bills WHERE category LIKE 'recommit';",engine)
print(bills['index'].size)
bills.head(3)

314


,index,category,chamber,congress,date,number,result,subjects,top_subject,type
0,55,recommit,h,113,2013-10-30 14:19:00,992,Failed,"{""Banking and financial institutions regulatio...",Finance and financial sector,hr
1,15,recommit,h,113,2013-02-06 11:52:00,444,Failed,"{""Budget deficits and national debt"",""Budget p...",Economics and public finance,hr
2,21,recommit,h,113,2013-09-19 17:59:00,3102,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr


In [48]:
bills = pd.read_sql("SELECT * FROM bills;",engine)
print(bills['index'].size)
bills.head(3)

5625


,index,category,chamber,congress,date,number,result,subjects,top_subject,type
0,0,amendment,h,113,2013-06-20 11:09:00,1947,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr
1,1,amendment,h,113,2013-11-21 11:22:00,1900,Failed,"{""Administrative law and regulatory procedures...",Energy,hr
2,2,amendment,h,113,2013-07-10 14:32:00,2609,Failed,"{""Administrative law and regulatory procedures...",Economics and public finance,hr


In [54]:
test = pd.unique(bills.number)
print(test)

[1947 1900 2609 ...,  735  764 4890]


In [58]:
#for i in range(0,test.size):
q = 'number == '+str(test[0])
foo = bills.query(q)

In [59]:
foo.head(5)

,index,category,chamber,congress,date,number,result,subjects,top_subject,type
0,0,amendment,h,113,2013-06-20 11:09:00,1947,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr
27,23,amendment,h,113,2013-06-20 11:18:00,1947,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr
56,50,amendment,h,113,2013-06-20 13:03:00,1947,Agreed to,"{""Administrative law and regulatory procedures...",Agriculture and food,hr
98,93,amendment,h,113,2013-06-20 10:50:00,1947,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr
102,97,amendment,h,113,2013-06-19 18:36:00,1947,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr


In [ ]:
#need to find amendment number and replace bill info for amendment votes with info from the amendments.